In [73]:
import tensorflow as tf
import tensorflow_text as tf_text
from random import randint
datafiles = "/home/ericjm24/fimarch/txt/*.txt"
dataset = tf.data.TextLineDataset(tf.data.Dataset.list_files(datafiles))
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, r'''([!(),.?\-'";:])''', r' \1 '))
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, r'''[^ a-z0-9A-Z!(),.?\-'";:]''', ''))
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, r'\s+', ' '))

In [2]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 50000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

vocab = bert_vocab.bert_vocab_from_dataset(
    dataset.batch(batch_size = 5, num_parallel_calls=tf.data.AUTOTUNE).prefetch(10),
    **bert_vocab_args
)

def write_vocab_file(filepath, vocab):
  with open(filepath, 'w') as f:
    for token in vocab:
      print(token, file=f)

write_vocab_file('vocab.txt', vocab)

In [74]:
tokenizer = tf_text.BertTokenizer('vocab.txt', **bert_tokenizer_params)

In [6]:
for item in dataset.take(1):
    print(tokenizer.tokenize(item).merge_dims(-2,-1).to_list())

[[111, 88, 567, 681, 52, 55, 523, 9, 56, 272, 25, 190, 3217, 314, 11, 88, 9, 169, 90, 57, 24, 33, 3218, 66, 57, 85, 116, 51, 523, 9, 79, 186, 11, 1476, 186, 9, 1638, 88, 567, 11, 33, 86, 3922, 66, 110, 5355, 6919, 11, 69, 57, 71, 125, 3506, 25, 241, 562, 11, 33, 59, 299, 781, 9, 79, 186, 11, 33, 113, 9, 186, 9, 79, 88, 567, 11, 53, 176, 294, 33, 212, 3743, 57, 9, 103, 57, 91, 52, 397, 122, 58, 244, 24, 191, 9, 88, 9, 186, 301, 64, 25, 245, 11, 88, 567, 53, 186, 244, 96, 25, 182, 61, 81, 71, 60, 51, 523, 9, 81, 392, 67, 25, 1284, 53, 88, 567, 284, 186, 76, 55, 2921, 59, 91, 11, 76, 59, 73, 5355, 1976, 91, 9, 88, 24, 131, 186, 11, 811, 207, 474, 80, 4085, 9, 186, 11, 88, 567, 120, 52, 51, 231, 11, 56, 59, 139, 25, 147, 1227, 61, 51, 150, 80, 51, 5355, 1976, 71, 110, 1094, 231, 11, 122, 474, 77, 58, 24, 122, 237, 33, 485, 24, 122, 102, 821, 52, 424, 102, 76, 156, 155, 91, 24, 88, 567, 278, 11, 119, 9, 51, 156, 1434, 917, 61, 160, 11, 51, 134, 5489, 80, 51, 156, 77, 61, 273, 1363, 1203, 60

In [75]:
WINDOW = 500
dataset = dataset.map(lambda x: tf.strings.split(x, ' '))
dataset = dataset.map(lambda x: tf_text.sliding_window(x, WINDOW)).unbatch()
dataset = dataset.map(lambda x: tf.strings.reduce_join(x, axis=0, separator = ' '))

In [87]:
MAX_TOKENS = WINDOW
def prepare_batch(words):
    temp = tokenizer.tokenize(words).merge_dims(-2,-1)
    input = temp[:,:MAX_TOKENS].to_tensor()
    out = temp[:,1:(MAX_TOKENS+1)].to_tensor()
    return input, out

In [88]:
BUFFER_SIZE = 200000
BATCH_SIZE = 256
def make_batches(ds):
    return (
        ds
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE)
        .map(prepare_batch, tf.data.AUTOTUNE)
        .prefetch(buffer_size = tf.data.AUTOTUNE)
    )

In [83]:
print(tokenizer.tokenize(dataset.batch(4)).merge_dims(-2, -1))

ValueError: Attempt to convert a value (<_BatchDataset element_spec=TensorSpec(shape=(None,), dtype=tf.string, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.batch_op._BatchDataset'>) to a Tensor.

In [89]:
for inp, outp in make_batches(dataset).take(1):
    print(inp.shape)
    print(outp.shape)

2023-07-06 19:11:53.669141: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 12371004000 exceeds 10% of free system memory.


(256, 500)
(256, 500)
